# Part 1

Import some of the libraries used

In [20]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

Get the Wiki page and parse it with BeautifulSoup

In [21]:
wiki_toronto = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').content, 'lxml')

Extract postal codes, boroughs and neighborhoods and put them in a DataFrame

In [29]:
def clean_content(content):
    content = content.replace(' / ', ', ')
    content = content.replace('(', '')
    content = content.replace(')', '')
    return content

items = []
for item in wiki_toronto.table.find_all('td'):
    postal_code = item.p.b.text
    if item.p.span.text == "Not assigned":
        continue
    parts = [it for it in item.stripped_strings if it not in "()"]
    borough = parts[1]
    neighborhood = ' '.join(parts[2:])
    
    items.append((postal_code, clean_content(borough), clean_content(neighborhood)))

df_toronto = pd.DataFrame([it for it in items], columns=['PostalCode', 'Borough', 'Neighborhood'])
df_toronto.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District , Ryerson"


In [30]:
df_toronto.shape

(103, 3)